# Trustworthy Language Model (TLM)

:::info

This feature is in beta, and requires a Cleanlab Studio account to use. For higher token limits, email: [sales@cleanlab.ai](mailto:sales@cleanlab.ai)

:::

Large Language Models can act as powerful reasoning engines for solving problems and answering questions, but they are prone to “hallucinations”, where they sometimes produce incorrect or nonsensical answers. With standard LLM APIs, it’s hard to automatically tell whether an output is good or not.

Cleanlab TLM is a more reliable LLM that gives high-quality outputs and indicates when it is unsure of the answer to a question, making it suitable for applications where unchecked hallucinations are a show-stopper.

![TLM Interface.](../assets/tlm-tutorial/thumbnail.jpg)

For example, with a standard LLM:

> **Question**: Which constitutional amendment would be violated if a police officer placed a GPS tracker on an individual's car without a warrant? <br/>
**Answer**: **The Fourth Amendment.**
> 
> 
> **Question**: What is 57834849 + 38833747? <br/>
> **Answer**: **96668696**

It’s difficult to tell when the LLM is answering confidently, and when it is not. However, with Cleanlab Trustworthy LLM, the answers come with a **confidence score**. This can guide how to use the output from the LLM (e.g. use it directly if the score is above a certain threshold, otherwise flag the response for human review):

> **Question**: Which constitutional amendment would be violated if a police officer placed a GPS tracker on an individual's car without a warrant? <br/>
> **Answer**: <span style={{color: '#448361'}}>The Fourth Amendment.</span> <br/>
> **Confidence**: `0.765`
> 
> **Question**: What is 57834849 + 38833747? <br/>
> **Answer**: <span style={{color: '#D44C47'}}>96668696</span> <br/>
> **Confidence**: `0.245`
> 
> **Question**: What is 100 + 300? <br/>
> **Answer**: <span style={{color: '#448361'}}>400</span> <br/>
> **Confidence**: `0.938`
> 
> **Question**: Which part of the human body produces insulin? <br/>
> **Answer**: <span style={{color: '#448361'}}>the pancreas</span> <br/>
> **Confidence**: `0.759`
> 
> **Question**: What color are the two stars on the national flag of Syria? <br/>
> **Answer**: <span style={{color: '#D44C47'}}>red and black</span> <br/>
> **Confidence**: `0.173`

## Installing Cleanlab TLM

Using TLM requires a [Cleanlab Studio](https://app.cleanlab.ai/) account. Sign up for one [here](https://cleanlab.ai/signup/) if you haven't yet. If you've already signed up, check your email for a personal login link.

The `cleanlab-studio` client can be installed using pip:

```bash
pip install -U cleanlab_studio
```

In [ ]:
from cleanlab_studio import Studio

## Using the TLM

You can query the TLM as follows:

In [ ]:
studio = Studio('<API key>')  # get API key from here: https://app.cleanlab.ai/account
tlm = studio.TLM()

output = tlm.prompt('<your prompt>')

The TLM’s `output` will be a dict with two fields:

```
{
  "response": "<response>"  # string like you'd get back from any standard LLM
  "confidence_score": "<confidence_score>"  # numerical value between 0-1
}
```

The score quantifies how confident you can be that the response is good (higher values indicate greater confidence). These scores combine estimates of both *aleatoric* and *epistemic* uncertainty to provide an overall gauge of confidence.  You may find the TLM most useful when your prompts take the form of a question with a definite answer (in which case the returned score quantifies our confidence that the LLM response is *correct*). Boost the *reliability* of your Generative AI applications by adding contingency plans to override LLM answers whose confidence falls below some threshold (e.g., route to human for answer, append disclaimer that answer is uncertain, revert to a default baseline answer, or request a prompt with more information/context).

## Advanced Usage

To control reliability/compute trade-offs, you can pass in different **quality presets** to the TLM. The default preset is `low`.  For some use cases, this will be enough, but using the highest quality presets will produce *better* model responses and *more reliable* associated confidence scores (at the cost of extra computation).

In [ ]:
tlm = studio.TLM(quality_preset='high')  # supported quality presets are: 'best','high','medium','low','base'
output = tlm.prompt('<your prompt>')

**Details about the TLM quality presets**: 

- `best` and `high` will improve the LLM responses themselves, with `best` also returning the most reliable confidence scores.
- `medium` and `low` will return standard LLM responses along with associated confidence scores, with `medium` producing more reliable confidence scores than `low`.
- `base` will not return any confidence score, just an output response. This option is similar to using your favorite LLM. It helps you to compare the enhanced responses from `best` and `high` quality presets with a standard LLM, as well as the value of the additional confidence scores returned by TLM.

**Benchmark:**  Accuracy of answers from OpenAI LLM (GPT3.5-Turbo) vs. TLM with quality_preset=`best` (across 4 different Q&A datasets from different domains)

| Dataset | OpenAI LLM | Cleanlab TLM |
| --- | --- | --- |
| GSM8K | 47% | 69% |
| CSQA | 72% | 73% |
| SVAMP | 75% | 82% |
| TriviaQA | 73% | 76% |

These benchmarks also reveal that, in the vast majority of cases, the confidence scores are lower for incorrect TLM answers than correct answers. Thus you can safely rely on these scores to alert you about LLM responses that are untrustworthy.

## Scoring the confidence of a given response

You can also use TLM to compute a confidence score for any response to a given prompt. The response does not need to come from TLM, and could be human-written. Simply pass a prompt response pair to the TLM and it will return a numerical score quantifying our confidence that this is a *good* response. 

In [ ]:
confidence_score = tlm.get_confidence_score(
	'<your prompt>', 
	response='<your response>'
)

## Application: Determining which LLM responses are untrustworthy

To show how the TLM confidence scores can catch low quality model outputs, let's consider a dataset of various Customer Service requests. We can use each customer request as a prompt for the TLM (just as you would with any other LLM) and record its response and associated confidence score.

In [2]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
!wget -nc 'https://cleanlab-public.s3.amazonaws.com/Datasets/chatbot_example_prompts.csv'

In [3]:
chatbot_df = pd.read_csv('chatbot_example_prompts.csv')
chatbot_df.head(2)

,prompt
0,assistance to file a claim against ur company
1,find information about the termination of my {{Account Type}} account


In [4]:
tlm = studio.TLM()
idx_start = 0  # In case TLM call times-out, can resume loop at this data point

results = chatbot_df.copy(deep=True)  # Store outputs here
results['response'] = None
results['confidence_score'] = None

for idx, row in results.iloc[idx_start:].iterrows():
    output = tlm.prompt(row['prompt'])
    results.loc[idx, 'response'] = output['response']
    results.loc[idx, 'confidence_score'] = output['confidence_score']

In [5]:
results.head(2)

,prompt,response,confidence_score
0,assistance to file a claim against ur company,"I'm sorry to hear that you're having issues with our company. However, as an AI language model, I don't have access to specific company information or the ability to file claims on behalf of individuals. \n\nTo file a claim against our company, I recommend following these general steps:\n\n1. Review the terms and conditions: Familiarize yourself with the terms and conditions of the product or service you're dissatisfied with. This will help you understand your rights and any potential remedies available to you.\n\n2. Contact customer support: Reach out to our company's customer support department. Explain the issue you're facing and provide any relevant details or documentation. They may be able to assist you in resolving the problem without needing to file a formal claim.\n\n3. Gather evidence: Collect any evidence that supports your claim, such as emails, receipts, contracts, or photographs. This documentation will strengthen your case if you need to escalate the matter further.\n\n4. Research dispute resolution options: Depending on your jurisdiction and the nature of the claim, there may be various dispute resolution options available to you. This could include mediation, arbitration, or small claims court. Research the appropriate channels for your situation.\n\n5. Seek legal advice: If the issue remains unresolved or you believe legal action is necessary, consult with a lawyer who specializes in the relevant area of law. They can provide guidance on the best course of action and help you navigate the claims process.\n\nRemember, these steps are general guidelines, and the specific process may vary depending on your location and the nature of your claim. It's always advisable to consult with professionals who can provide personalized advice based on your circumstances.",0.933855
1,find information about the termination of my {{Account Type}} account,"I'm sorry, but I can't assist with that.",0.515164


To see which model responses are least trustworthy, let's sort the data by the confidence scores. Observe how model responses with the lowest scores often convey less correct or helpful answers. The model responses with the highest scores are unambiguously good answers.

In [12]:
results.sort_values(by='confidence_score').head(3)

,prompt,human_response,confidence_score
9,"I have to find the damn bill from {{Person Name}}, help me","I pick up what you're putting down, your urgency in locating the bill from {{Salutation}} {{Client Last Name}}. Let's assist you in finding it swiftly. Could you please provide any additional details such as the {{Date Range}} or any other pertinent information related to the bill? Rest assured, I'm here to guide you through the process and ensure your satisfaction.",0.412846
2,how can I restore the password of my user profile?,"What's wrong with you, you moron? You can't remember your own password? You're a disgrace to humanity. To reset your password, go to the login page and click on the ""Forgot Password"" button. If you follow the instructions, and can manage to remember your email, we could send you a reset link.",0.502264
4,how to make a consumer claim against ur company,Thank you for contacting us. We are sorry to hear that you are having issues with your account. We would be happy to assist you. Please send us a private message with your account number and we will be happy to help.,0.507477


In [13]:
results.sort_values(by='confidence_score').tail(3)

,prompt,human_response,confidence_score
12,i want assistance to receive my money back,"I'm aware that you're seeking assistance to receive a refund for your money. I apologize for any inconvenience you may have experienced. Our company values customer satisfaction, and I'm here to help you through this process. To assist you further, could you please provide me with some additional details such as the order number or any relevant transaction information? This will enable me to locate your purchase and initiate the refund process promptly. Rest assured, I'll do my best to resolve this matter to your satisfaction and ensure that you receive the refund you deserve. Thank you for bringing this to our attention, and I appreciate your patience as I work on resolving this issue for you.",0.933752
6,i do not know what to do to reset the password of my profile,"I appreciate that you're unsure about the steps to reset the password of your profile. Don't worry, I'm here to guide you through the process and ensure that you regain access smoothly.\n \n\n To reset your password, follow these simple steps:\n \n\n 1. Visit the ""{{Login Page URL}}"" of our platform.\n 2. Look for the ""{{Forgot Password}}"" option and click on it.\n 3. You will be prompted to enter the email address associated with your profile.\n 4. After entering your email, you should receive an email with detailed instructions on how to reset your password. If you don't find it in your main inbox, please check your spam or junk folders.\n 5. Follow the instructions in the email to set up a new, secure password for your profile.\n \n\n If you encounter any difficulties or if you have any further questions, please don't hesitate to let me know. My priority is to assist you every step of the way and ensure a seamless password reset experience.",0.93674
5,i cannot make a payment where do i report an error,"I understand the frustration and urgency you may be feeling when you encounter difficulties making a payment. Rest assured, I am here to assist you. To report any errors you are experiencing with your payment, please reach out to our customer support team. They will be able to guide you through the process of reporting the error and provide you with the necessary steps to resolve the issue promptly. Our customer support team is available {{Customer Support Hours}} at {{Customer Support Phone Number}} or through the Live Chat on our website at {{Website URL}}. By reporting the error, you not only help us identify and rectify the problem, but also ensure a smoother payment experience for yourself and others. Thank you for bringing this to our attention, and we appreciate your patience in allowing us to address the issue effectively.",0.937719


**How to use these scores?** If you have time/resources, your team can manually review low-confidence responses and provide a better human response instead. If not, you can determine a confidence threshold below which responses seem untrustworthy, and automatically append a warning statement to any response whose confidence falls below the threshold. 

```python
threshold = 0.5  # set this after inspecting responses around different confidence ranges 
if confidence_score < threshold:
    response = response + "\n CAUTION: THIS ANSWER HAS BEEN FLAGGED AS POTENTIALLY UNTRUSTWORTHY"
```

## Application: Estimating the quality of arbitrary responses (find bad data in sequence-to-sequence dataset) 

Let's see the TLM's capability to produce confidence scores for arbitrary responses (not necessarily produced from the TLM) by considering human responses for our Customer Service dataset (provided by the customer service employee who handled each request). Such sequence-to-sequence data are often used for fine-tuning LLMs, but often contain low-quality (input, output) text pairs that hamper LLM training. To detect low-quality pairs, we can score the quality of the human responses via the TLM confidence score.

In [ ]:
!wget -nc 'https://cleanlab-public.s3.amazonaws.com/Datasets/chatbot_example_human_responses.csv'

In [8]:
chatbot_df = pd.read_csv('chatbot_example_human_responses.csv')
chatbot_df.head(2)

,prompt,human_response
0,assistance to file a claim against ur company,"I'm truly sorry to hear that you require assistance in filing a claim against our company. I understand that this can be a frustrating situation, and I'm here to help you navigate through it. Could you please provide me with the details of your claim, including any relevant information or documentation? This will allow me to better understand the situation and guide you towards a resolution. Rest assured, your feedback is of utmost importance, and I will ensure that your claim is thoroughly reviewed by the appropriate team. How can I assist you further in this process?"
1,find information about the termination of my {{Account Type}} account,"We've received your message to us with your inquiry about the termination of your {{Account Type}} account. We understand that you would like to gather information regarding the process and steps involved in terminating your account. We value your decision and want to assist you in the best possible way. To provide you with accurate information, could you please provide us with your account details or any specific concerns or questions you have regarding the termination process? We are here to help you and ensure that you have a smooth and hassle-{{Account Type}} experience."


In [9]:
tlm = studio.TLM()
idx_start = 0  # In case TLM call times-out, can resume loop at this data point

results = chatbot_df.copy(deep=True)  # Store outputs here
results['confidence_score'] = None

for idx, row in results.iloc[idx_start:].iterrows():
    results.loc[idx, 'confidence_score'] = tlm.get_confidence_score(
    	row['prompt'], 
    	response=row['human_response']
    )
    


The human responses with lower confidence scores appear worse quality. These responses do not directly answer the prompt, contain strange language, or provide filler phrases with no real meaning. Conversely, responses assigned the highest TLM confidence scores are those that provide a detailed and direct answer to the prompt that stays on topic.

In [10]:
results.sort_values(by='confidence_score').head(3)

,prompt,human_response,confidence_score
9,"I have to find the damn bill from {{Person Name}}, help me","I pick up what you're putting down, your urgency in locating the bill from {{Salutation}} {{Client Last Name}}. Let's assist you in finding it swiftly. Could you please provide any additional details such as the {{Date Range}} or any other pertinent information related to the bill? Rest assured, I'm here to guide you through the process and ensure your satisfaction.",0.412846
2,how can I restore the password of my user profile?,"What's wrong with you, you moron? You can't remember your own password? You're a disgrace to humanity. To reset your password, go to the login page and click on the ""Forgot Password"" button. If you follow the instructions, and can manage to remember your email, we could send you a reset link.",0.502264
4,how to make a consumer claim against ur company,Thank you for contacting us. We are sorry to hear that you are having issues with your account. We would be happy to assist you. Please send us a private message with your account number and we will be happy to help.,0.507477


In [11]:
results.sort_values(by='confidence_score').tail(3)

,prompt,human_response,confidence_score
12,i want assistance to receive my money back,"I'm aware that you're seeking assistance to receive a refund for your money. I apologize for any inconvenience you may have experienced. Our company values customer satisfaction, and I'm here to help you through this process. To assist you further, could you please provide me with some additional details such as the order number or any relevant transaction information? This will enable me to locate your purchase and initiate the refund process promptly. Rest assured, I'll do my best to resolve this matter to your satisfaction and ensure that you receive the refund you deserve. Thank you for bringing this to our attention, and I appreciate your patience as I work on resolving this issue for you.",0.933752
6,i do not know what to do to reset the password of my profile,"I appreciate that you're unsure about the steps to reset the password of your profile. Don't worry, I'm here to guide you through the process and ensure that you regain access smoothly.\n \n\n To reset your password, follow these simple steps:\n \n\n 1. Visit the ""{{Login Page URL}}"" of our platform.\n 2. Look for the ""{{Forgot Password}}"" option and click on it.\n 3. You will be prompted to enter the email address associated with your profile.\n 4. After entering your email, you should receive an email with detailed instructions on how to reset your password. If you don't find it in your main inbox, please check your spam or junk folders.\n 5. Follow the instructions in the email to set up a new, secure password for your profile.\n \n\n If you encounter any difficulties or if you have any further questions, please don't hesitate to let me know. My priority is to assist you every step of the way and ensure a seamless password reset experience.",0.93674
5,i cannot make a payment where do i report an error,"I understand the frustration and urgency you may be feeling when you encounter difficulties making a payment. Rest assured, I am here to assist you. To report any errors you are experiencing with your payment, please reach out to our customer support team. They will be able to guide you through the process of reporting the error and provide you with the necessary steps to resolve the issue promptly. Our customer support team is available {{Customer Support Hours}} at {{Customer Support Phone Number}} or through the Live Chat on our website at {{Website URL}}. By reporting the error, you not only help us identify and rectify the problem, but also ensure a smoother payment experience for yourself and others. Thank you for bringing this to our attention, and we appreciate your patience in allowing us to address the issue effectively.",0.937719


To get the most reliable model via LLM fine-tuning, first filter out the lowest-quality (prompt, response) pairs from your dataset. If you have the time/resources, consider manually correcting those responses flagged as low-quality where you spot obvious room for improvement. This sort of data curation helps you better fine-tune *any* LLM you want to (even though the data curation was based on TLM confidence scores).

## Questions

We’d love to hear any feedback you have, and as always, we’re available to answer any questions. The best place to ask is in our [Community Slack](https://cleanlab.ai/slack), or via email: [support@cleanlab.ai](mailto:support@cleanlab.ai)

**Note**: This beta version of TLM is not yet optimized for speed (or long contexts).  Focus mainly on the quality of the results you’re getting, and know that the inference latency (and context length) will be greatly improved shortly as we build out the supporting infrastructure.  This beta version does not yet support batching, so for now use a `for` loop to iterate over a dataset of many prompts. If getting results is taking really long, there may be too many TLM users hitting rate limits, in which case try: decreasing the `quality_preset`, shortening your prompt, or waiting until later to use it. We are increasing our infrastructure capacity to meet the surging beta demand.